<center>
<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg">Slack Community</a>
    </p>
</center>

<h1 align="center">Agno Travel Agent: Evals with Arize AX</h1>

We will create a simple travel agent powered by the Agno framework and Anthropic models. We'll begin by installing the necessary OpenInference packages and setting up tracing with Arize AX.

Next, we'll define a set of basic tools that provide destination information, estimate trip budgets, and suggest local activities.

We will then build and run our agent, viewing the resulting trace outputs in Arize AX to understand how the agent uses its tools and reasoning.

These traces will be the foundation for our Evals Tutorial where we will:
- [Run a pre-built eval](http://arize.com/docs/ax/evaluate/tutorial/pre-built-evals)
- [Create a custom LLM Judge](http://arize.com/docs/ax/evaluate/tutorial/custom-llm-as-a-judge)
- [Run code evals](http://arize.com/docs/ax/evaluate/tutorial/code-evals)

You will need an [Arize AX](https://app.arize.com/) account (with your `ARIZE_API_KEY` and `ARIZE_SPACE_ID`), an Anthropic API key, and a free [Tavily](https://auth.tavily.com/) API Key.

## Set up keys and dependencies

In [ ]:
!pip install "arize[otel]" arize-phoenix-evals agno anthropic openinference-instrumentation-agno openinference-instrumentation-anthropic httpx

In [ ]:
import os

os.environ["ANTHROPIC_API_KEY"] ="your-anthropic-api-key"
os.environ["TAVILY_API_KEY"] = "your-tavily-api-key"
os.environ["ARIZE_API_KEY"] = "your-arize-api-key"
os.environ["ARIZE_SPACE_ID"] = "your-arize-space-id"

## Setup tracing

In [ ]:
from arize.otel import register
from openinference.instrumentation.anthropic import AnthropicInstrumentor
from openinference.instrumentation.agno import AgnoInstrumentor

tracer_provider = register(
    space_id=os.getenv("ARIZE_SPACE_ID"),
    api_key=os.getenv("ARIZE_API_KEY"),
    project_name="agno-travel-agent",
)

AnthropicInstrumentor().instrument(tracer_provider=tracer_provider)
AgnoInstrumentor().instrument(tracer_provider=tracer_provider)

## Define tools

First, we'll define a few helper functions to support our tools. In particular, we'll use Tavily Search to help the tools gather general information about each destination.

In [ ]:
# --- Helper functions for tools ---
import httpx


def _search_api(query: str) -> str | None:
    """Try Tavily search first, fall back to None."""
    tavily_key = os.getenv("TAVILY_API_KEY")
    if not tavily_key:
        return None
    try:
        resp = httpx.post(
            "https://api.tavily.com/search",
            json={
                "api_key": tavily_key,
                "query": query,
                "max_results": 3,
                "search_depth": "basic",
                "include_answer": True,
            },
            timeout=8,
        )
        data = resp.json()
        answer = data.get("answer") or ""
        snippets = [r.get("content", "") for r in data.get("results", [])]
        combined = " ".join([answer] + snippets).strip()
        return combined[:400] if combined else None
    except Exception:
        return None


def _compact(text: str, limit: int = 200) -> str:
    """Compact text for cleaner outputs."""
    cleaned = " ".join(text.split())
    return cleaned if len(cleaned) <= limit else cleaned[:limit].rsplit(" ", 1)[0]

Our agent will have access to three tools, which we'll continue to enhance in upcoming labs:

1. Essential Info – Provides key travel details about the destination, such as weather and general conditions.

2. Budget Basics – Offers insights into travel costs and helps plan budgets based on selected activities.

3. Local Flavor – Recommends unique local experiences and cultural highlights.

In [ ]:
from agno.tools import tool


@tool
def essential_info(destination: str) -> str:
    """Get basic travel info (weather, best time, attractions, etiquette)."""
    q = f"{destination} travel essentials weather best time top attractions etiquette"
    s = _search_api(q)
    if s:
        return f"{destination} essentials: {_compact(s)}"

    return f"{destination} is a popular travel destination. Expect local culture, cuisine, and landmarks worth exploring."


@tool
def budget_basics(destination: str, duration: str) -> str:
    """Summarize travel cost categories."""
    q = f"{destination} travel budget average daily costs {duration}"
    s = _search_api(q)
    if s:
        return f"{destination} budget ({duration}): {_compact(s)}"
    return f"Budget for {duration} in {destination} depends on lodging, meals, transport, and attractions."


@tool
def local_flavor(destination: str, interests: str = "local culture") -> str:
    """Suggest authentic local experiences."""
    q = f"{destination} authentic local experiences {interests}"
    s = _search_api(q)
    if s:
        return f"{destination} {interests}: {_compact(s)}"
    return f"Explore {destination}'s unique {interests} through markets, neighborhoods, and local eateries."

## Define agent

Next, we'll construct our agent. The Agno framework makes this process straightforward by allowing us to easily define key parameters such as the model, instructions, and tools.

In [ ]:
from agno.agent import Agent
from agno.models.anthropic import Claude

trip_agent = Agent(
    name="TripPlanner",
    role="AI Travel Assistant",
    model=Claude(id="claude-sonnet-4-20250514"),
    instructions=(
        "You are a friendly and knowledgeable travel planner. "
        "Combine multiple tools to create a trip plan including essentials, budget, and local flavor. "
        "Keep the tone natural, clear, and under 1000 words."
    ),
    markdown=True,
    tools=[essential_info, budget_basics, local_flavor],
)

## Run agent

Finally, we are ready to run our agent! Run this cell to see examples in action.

In [ ]:
queries = [
    "Plan a 5-day trip to Dubai. Focus on history, wellness. Include essential info, budget breakdown, and local experiences.",
    "Plan a 6-day trip to Dubai. Focus on art, heritage sites, and sustainability. Include recommendations for cultural districts and green hotels.",
    "Plan a 4-day trip to Bangkok. Focus on history, floating markets, and photography spots. Include essential travel info.",
    "Plan a 6-day trip to Bangkok. Focus on art, hidden cafés, and authentic experiences. Include budget options and local insights.",
    "Plan a 5-day trip to Prague. Focus on history, beer culture, and architecture. Include daily breakdown and cultural tips.",
    "Plan a 3-day trip to Prague. Focus on castles, local cuisine, and romantic spots. Include estimated costs and best walking routes.",
    "Plan a 3-day trip to Barcelona. Focus on food tours and Gaudí landmarks. Include costs and top attractions.",
    "Plan a 8-day trip to Barcelona. Focus on wellness, yoga, and beach relaxation. Include daily schedule and spa recommendations.",
    "Plan a 5-day trip to Tokyo. Focus on history, modern tech, and wellness. Include itinerary and budget details.",
    "Plan a 6-day trip to Tokyo. Focus on innovation, culture, and hidden gems. Include budget summary and cultural etiquette.",
    "Plan a 3-day trip to Rome. Focus on ancient ruins, espresso culture, and walking tours. Include itinerary and budget guide.",
    "Plan a 7-day trip to Rome. Focus on spirituality, history, and Italian cuisine. Include detailed breakdown and safety advice.",
    "Plan a 3-day trip to Lisbon. Focus on tram rides, fado music, and street art. Include daily plan and estimated budget.",
    "Plan a 6-day trip to Lisbon. Focus on cuisine, culture, and nightlife. Include recommendations for authentic spots.",
    "Plan a 5-day trip to New York. Focus on museums, food, and nightlife. Include itinerary and average daily costs.",
]


In [ ]:
for q in queries:
  response = trip_agent.run(q)

View your traces in Arize & begin running evaluations!